# A computational framework for the efficient formulation and calibration of large-scale, thermodynamically consistent kinetic models and its application

## Abstract

## Introduction

Cells use signal transduction to respond to extracellular cues. Extracellular ligands bind to RTKs that initiate phosphorylation cascades. Depending on protein abundance, different protein complexes will be assembled, which determines which part of downstream signaling will be activated. In contrast to phosphorylation, assembly of protein complexes does not consume energy. Consumption of energy can be checked via wegscheider cycle conditions. Enumerating all cycles is an NP-hard problem, so structured approach is necessary to ensure energy conservation. Faeder et al have introduced thermodynamic rule based modelling which derives kinetic rates from the specified gibbs free energies. Binding reactions generated using thermodynamic framework inherently satisfy energy conservation, making them biophysically more plausible. Also allows specification of rules that consume energy, such as phosphorylation or dephosphorylation by explicitely specifying consumption of cellular energy sources such as ATP or NADPH.
Derivation of kinetic rates from gibbs free energies are carried out using arrhenius equation, but more modern formulations exist. Formulation provides an intuition why log-scale is more natural scale for most parameters and why optimization etc works better when log-transformed.

Energy conservation and corresponding wegscheider conditions introduce an constraint on parameters, which may suggest that this yields smaller less complex models. However, thermodynamically sounds models require reversibility of all reactions, which usually leads to more complex models. Specifically, binarization of reactions usually leads to non-reversible reactions that may violate energy conservation. An example for binarization of a reaction would be binding of RAS. RAF has a much higher affinity to gtp loaded Ras (RAS-gtp) compared to gdp loaded Ras (RAS-gdp), so it is tempting to implement a conditional reaction where RAF only binds RAS-gtp, but not RAS-gdp. As RAS is a GTPase, it can hydrolyse the loaded gtp to gdp, converting RAS-gtp to RAS-gdp. This reaction creates a complex in which RAS-gdp is bound to RAF. As the corresponding binding reaction was explicitely not created, the unbinding reaction for RAS-gdp and RAF would necessarily be irreversible and may thus violate energy conservation. Implementing a non-binarized reaction where RAF binds both RAS-gtp and RAS-gdp can easily be done in the thermodynamic framework, where the difference in affinity would be implemented as difference in Gibbs Free Energy, and would be guaranteed to satisfy wegscheider cycle conditions and thus satisfy energy conservation. However, the non-binarized, thermodynamic model needs to account for more species, which may pose computational challenges when the model describes more complex oligomerization processes. 

Tools such as NFSim are particularly suited to simulate 

## Formulating a Thermodynamic Model of RAF inhibition in PySB

First import various classes that are necessary to define rules and initialize the model

In [1]:
from pysb import Model, Compartment, Rule, EnergyPattern, Expression
model = Model('thermo_raf');

### Protein Species
As first step of the model construction, we have to define all molecules which we want to account for. In PySB, this is done through using the Monomer class. Each monomer is initialized by a unique identifier, a list of site as well as, optionally, a list of states for each site. Sites without states are typically describe the bonds involved in complex formation, while sites with states can also describe (post-translational) protein modifitations such as mutations or phosphorylation. 

Here, we we will define monomers _BRAF_ and _CRAF_ which both feature the interaction sites _RBD_ (interaction domain with RAS), _mek_ (interaction domain with MEK), _raf_ (dimerization domain) and _rafi_ (inhibitor binding site). For BRAF we also add a mutation sites for the 600th amino acid, which can take values _V_ (valine, wildtype) or _E_ (glutamic acid, oncogenic mutation).

In [2]:
from pysb import Monomer
Monomer('BRAF', ['AA600', 'RBD', 'mek', 'raf', 'rafi'], {'AA600': ['E', 'V']})
Monomer('CRAF', ['RBD', 'mek', 'raf', 'rafi']);

Next, we will specify initial concentrations. For this model, the concentration of all molecular species will be in $\mu\mathrm{M}$. However, we will use protein counts per cell derived from absolute proteomics to inform initial conditions. Accordingly, parameters that define initial concentrations have to be transformed from molecule per cell to $\mu\mathrm{M}$, which is achieved by dividing them by cell volume (here assumed to be $1pL = 10^{-12} L$) and Avogadro constant ($6.022 10^{23}$ molecules) and multiplying with $10^{6}$ to account for the unit prefix $\mu$.

In the following code we first introduce two expressions that define the avogadro constant and cell volume. PySB automatically creates workspace variables with the name of the expressions, which is passed as first argument, that can then later be used as argument to functions or as part of more complex expressions. Next, we introduce two parameter _BRAF_0_ and _CRAF_0_ that define initial abundances as molecules per cell and convert these abundances to concentrations in $\mu\mathrm{M}$ using the expressions _initBRAF_ and _initCRAF_. These expressions are then used to define the initial abundances of two molecular species for CRAF and BRAF. A molecular species is defined by a pattern, which is created by invoking the respective monomer with the state of each site as keyword argument. For initial conditions, the respective patterns have to be explicit, i.e., all states have to defined. Here, we specify that all sites are unbound (denoted by _None_), with the exception of the the _AA600_ site, for which we have to pick ony of the previously specified states, where we pick the oncogenic variant denoted by _E_.

In [3]:
from pysb import Parameter, Expression, Initial
import sympy as sp

# define Avogadro constant and volume as hardcoded expressions
Expression('N_Avogadro', sp.Float(6.02214076e+23))
Expression('volume', sp.Float(1e-12))

# define initial abundance parameters 
Parameter('BRAF_0', 0.0)
Parameter('CRAF_0', 0.0)

# convert initial abundances to concentrations
Expression('initBRAF', 1000000.0*BRAF_0/(N_Avogadro*volume))
Expression('initCRAF', 1000000.0*CRAF_0/(N_Avogadro*volume))

# define initial molecular species
Initial(BRAF(AA600='E', RBD=None, mek=None, raf=None, rafi=None), initBRAF)
Initial(CRAF(RBD=None, mek=None, raf=None, rafi=None), initCRAF);

### Protein Interactions
In rule-based modeling, all dynamic interactions are specified as rules. Rules are generalizations of biochemical reactions and their action is defined by a reactant pattern and a product pattern. When applied to a molecular species, the action of the rule, i.e., the respective biochemical reaction, is implemented by replacing one occurence of the reactant pattern by the product pattern in a molecular species. The reactant pattern and product pattern do not have to be explicit and a single rule can define multiple different biochemical reactions, depending in how often and in how many molecular species the reactant pattern occurs.

Non-thermodynamic rules additionally require forward and, if applicable, reverse reaction rates. In contrast, thermodynamic rules are specified in terms of activation energy and the phenomenological constant $\phi$, which encodes whether changes in free energy affect forwards or reverse rates. For thermodynamic rules, the reaction rate also depends on EnergyPatterns that may apply to educts or products of the reaction. This means that all reactions generated by non-thermodynamic rules have the same reaction rates, unless the rate is a local function. In contrast, reactions generated by thermodynamic rules can have context dependent reaction rates.

In the following we will specify the dimerization of RAF molecules. We introduce three parameters, the forward rate $k_f$, the dissociation constant $k_D$ and the thermodynamic parameter $\phi$. 

In [4]:
Parameter('bind_RAF_RAF_Ea', 10.0)
Parameter('bind_RAF_RAF_dG', 0.01)
Parameter('bind_RAF_RAF_phi', 1.0)

Expression('Ea0_bind_RAF_RAF', -bind_RAF_RAF_phi*bind_RAF_RAF_dG - bind_RAF_RAF_Ea);

To construct homo- and heterodimerization rules for all RAF paralogs, we use itertools to loop over all combinations of BRAF and CRAF. This assumes equal affinities and dynamics for all proteins.

In [5]:
import itertools as itt

for RAF1, RAF2 in itt.combinations_with_replacement([BRAF, CRAF], 2):
    Rule(f'{RAF1.name}_and_{RAF2.name}_bind_and_dissociate', 
         RAF1(raf=None) + RAF2(raf=None) | RAF1(raf=1) % RAF2(raf=1), 
         bind_RAF_RAF_phi, Ea0_bind_RAF_RAF, energy=True)
    EnergyPattern(f'ep_bind_{RAF1.name}_{RAF2.name}', 
                  RAF1(raf=1) % RAF2(raf=1), bind_RAF_RAF_dG);

### RAF inhibitor
Here we introduce a RAF inhibitor RAFi. We assume that the inhibitor is added to the cell medium at some point and quickly diffuses in and out of the cell. As extracellular space is much bigger than the volume of a cell, we can assume an infinite reservoir of molecules on the outside and assume that the intracellular inhibitors concentration will be unaffected by intracellular reactions. Accordingly, we specify the respective initial as constant, which means that the respective molecular species can participate in reactions, but it's concentration will remain constant.

In [6]:
Monomer('RAFi', ['raf'])

Initial(RAFi(raf=None), Parameter('RAFi_0', 0.0), fixed=True);

Next we define binding reactions for RAFi with both BRAF and CRAF, again assuming the same affinities and activation energies for both reactions.

In [7]:
Parameter('bind_RAFi_RAF_Ea', 10.0)
Parameter('bind_RAFi_RAF_dG', 0.01)
Parameter('bind_RAFi_RAF_phi', 1.0)
Expression('Ea0_bind_RAFi_RAF', -bind_RAFi_RAF_phi*bind_RAFi_RAF_dG - bind_RAFi_RAF_Ea)

for RAF in [BRAF, CRAF]:
    Rule(f'RAFi_and_{RAF.name}_bind_and_dissociate', 
         RAFi(raf=None) + RAF(rafi=None) | RAFi(raf=1) % RAF(rafi=1), 
         bind_RAFi_RAF_phi, Ea0_bind_RAFi_RAF, energy=True)
    EnergyPattern(f'ep_bind_{RAF.name}_RAFi', RAFi(raf=1) % RAF(rafi=1), bind_RAFi_RAF_dG);

### Paradoxical Activation
RAF inhibitors inhibit growth for BRAF mutants but promote growth for NRAS mutants. At the strucural level, this can be rationalized by assuming that RAF inhibitors have higher affinity towards drug-unbound RAF dimers and lower affinity towards drug bound RAF dimers. The symmetry conveyed by the no energy consuming nature of molecular binding reactions, implies that RAF inhibitors, at low to medium concentrations, promote dimerization but incompletely inhibit signaling as they only bind to one of the two protomers in a dimer. As MAPK signaling in NRAS mutant cells is mediated by RAF dimers, respective signaling is amplified, leading to increased growth. In the thermodynamic framework, this can be implemented by adding an energy pattern that controls the Gibbs free energy of RAF-RAF-RAFi trimers. Note that we do not specify how these trimers are formed, so the change in energy will equally apply to the rates of all reactions that produce these trimers. In this example, an decrease in energy would equally increase RAF dimerization and inhbitor binding, thus implementing the previously described symmetry.

In [8]:
Parameter('ep_RAF_RAF_mod_RAFi_single_ddG', 0.001)
for RAF1, RAF2 in itt.product([BRAF, CRAF], repeat=2):
    EnergyPattern(f'ep_{RAF1.name}_{RAF2.name}_mod_RAFi_single', 
                  RAF1(raf=1, rafi=None) % RAF2(raf=1, rafi=2) % RAFi(raf=2), ep_RAF_RAF_mod_RAFi_single_ddG);

To implement the protection of the second dimer, we add additional energy patterns that change the Gibbs free energy of RAFi-RAF-RAF-RAFi quatramers.

In [9]:
Parameter('ep_RAF_RAF_mod_RAFi_double_ddG', 1000.0)
for RAF1, RAF2 in itt.combinations_with_replacement([BRAF, CRAF], r=2):
    EnergyPattern(f'ep_{RAF1.name}_{RAF2.name}_mod_RAFi_double', 
                  RAFi(raf=2) % BRAF(raf=1, rafi=2) % BRAF(raf=1, rafi=3) % RAFi(raf=3), 
                  ep_RAF_RAF_mod_RAFi_double_ddG);

We will now load the remainder of the model from a file. Briefly, the full model describes MEK and ERK phosphorylation downstream as well as EGF stimulatable EGFR signaling upstream of RAF signaling. Moreover, it incorporates negative feedback from ERK on both MAPK and EGFR signaling. A comprehensive description of this part of the model is available in REF. Note however, that parts of the model were substantially simplified account for the reduced set of experimental data.

In [10]:
from pysb import Observable, ANY
def extend_model():
    Monomer('RAS', ['raf', 'state'], {'state': ['gdp', 'gtp']})
    Monomer('MEK', ['phospho'], {'phospho': ['p', 'u']})
    Monomer('ERK', ['phospho'], {'phospho': ['p', 'u']})
    Monomer('DUSP', [])

    Parameter('ep_RAF_RAF_mod_RASstategtp_double_ddG', 1000.0)
    Parameter('bind_RASstategtp_RAF_Ea', 10.0)
    Parameter('bind_RASstategtp_RAF_dG', 0.01)
    Parameter('bind_RASstategtp_RAF_phi', 1.0)
    Parameter('catalyze_RAF_RAFrafiNone_MEK_p_kcat', 10.0)
    Parameter('catalyze_RAFrafiNone_MEK_p_kcat', 10.0)
    Parameter('catalyze_MEKmekiNone_phosphop_ERK_p_kcat', 10.0)
    Parameter('synthesize_ERKphosphop_DUSP_ERK_gexpslope', 1000.0)
    Parameter('synthesize_ERKphosphop_DUSP_kdeg', 10.0)
    Parameter('synthesize_ERKphosphop_DUSP_ERK_kM', 0.0)
    Parameter('inhibition_ERKphosphop_RAS_kM', 0.0)
    Parameter('catalyze_PP2A_MEK_u_kcatr', 1.0)
    Parameter('catalyze_DUSP_ERK_u_kcatr', 1.0)
    Parameter('activation_RAS_kcat', 1.0)

    Parameter('RAS_0', 0.0)
    Parameter('MEK_0', 0.0)
    Parameter('ERK_0', 0.0)
    Parameter('EGF_0', 0.0)
    Parameter('DUSP_eq', 10000.0)

    Expression('Ea0_bind_RASstategtp_BRAF', 
               -bind_RASstategtp_RAF_phi*bind_RASstategtp_RAF_dG - bind_RASstategtp_RAF_Ea)
    Expression('Ea0_bind_RASstategtp_CRAF', 
               -bind_RASstategtp_RAF_phi*bind_RASstategtp_RAF_dG - bind_RASstategtp_RAF_Ea)
    
    Expression('catalyze_PP2A_MEK_u_kcat', catalyze_PP2A_MEK_u_kcatr*catalyze_RAFrafiNone_MEK_p_kcat)
    Expression('catalyze_DUSP_ERK_u_kcat', catalyze_DUSP_ERK_u_kcatr*catalyze_MEKmekiNone_phosphop_ERK_p_kcat)

    Expression('initMEK', 1000000.0*MEK_0/(N_Avogadro*volume))
    Expression('initERK', 1000000.0*ERK_0/(N_Avogadro*volume))
    Expression('initRAS', 1000000.0*RAS_0/(N_Avogadro*volume))

    Observable('modulation_ERKphosphop', ERK(phospho='p'))

    Expression('synthesize_ERKphosphop_DUSP_ERK_kmod', modulation_ERKphosphop*synthesize_ERKphosphop_DUSP_ERK_gexpslope/(modulation_ERKphosphop + synthesize_ERKphosphop_DUSP_ERK_kM) + 1)
    Expression('synthesize_ERKphosphop_DUSP_ksyn', 1.0*DUSP_eq*synthesize_ERKphosphop_DUSP_kdeg*synthesize_ERKphosphop_DUSP_ERK_kmod)
    Expression('inhibition_ERKphosphop_RAS_kmod', activation_RAS_kcat/(1 + modulation_ERKphosphop/inhibition_ERKphosphop_RAS_kM)*EGF_0/100.0)


    Rule('RASgtp_and_BRAF_bind_and_dissociate', 
         RAS(raf=None) + BRAF(RBD=None) | 
         RAS(raf=1) % BRAF(RBD=1), 
         bind_RASstategtp_RAF_phi, Ea0_bind_RASstategtp_BRAF, energy=True)
    Rule('RASgtp_and_CRAF_bind_and_dissociate', 
         RAS(raf=None) + CRAF(RBD=None) | 
         RAS(raf=1) % CRAF(RBD=1), 
         bind_RASstategtp_RAF_phi, Ea0_bind_RASstategtp_CRAF, energy=True)
    Rule('BRAF_BRAF_phosphorylates_MEK', 
         MEK(phospho='u') + BRAF(RBD=ANY, raf=2, rafi=None) % BRAF(RBD=ANY, raf=2) >> 
         MEK(phospho='p') + BRAF(RBD=ANY, raf=2, rafi=None) % BRAF(RBD=ANY, raf=2), 
         catalyze_RAF_RAFrafiNone_MEK_p_kcat)
    Rule('CRAF_BRAF_phosphorylates_MEK', 
         MEK(phospho='u') + CRAF(RBD=ANY, raf=2, rafi=None) % BRAF(RBD=ANY, raf=2) >> 
         MEK(phospho='p') + CRAF(RBD=ANY, raf=2, rafi=None) % BRAF(RBD=ANY, raf=2), 
         catalyze_RAF_RAFrafiNone_MEK_p_kcat)
    Rule('CRAF_CRAF_phosphorylates_MEK', 
         MEK(phospho='u') + CRAF(RBD=ANY, raf=2, rafi=None) % CRAF(RBD=ANY, raf=2) >> 
         MEK(phospho='p') + CRAF(RBD=ANY, raf=2, rafi=None) % CRAF(RBD=ANY, raf=2), 
         catalyze_RAF_RAFrafiNone_MEK_p_kcat)
    Rule('BRAFV600E_phosphorylates_MEK_bound1', 
         MEK(phospho='u') + BRAF(AA600='E', raf=None, rafi=None) >> 
         MEK(phospho='p') + BRAF(AA600='E', raf=None, rafi=None), 
         catalyze_RAFrafiNone_MEK_p_kcat)
    Rule('BRAFV600E_phosphorylates_MEK_bound2', 
         MEK(phospho='u') + BRAF(AA600='E', RBD=None, raf=ANY, rafi=None) >> 
         MEK(phospho='p') + BRAF(AA600='E', RBD=None, raf=ANY, rafi=None), 
         catalyze_RAFrafiNone_MEK_p_kcat)
    Rule('BRAFV600E_phosphorylates_MEK_bound3', 
         MEK(phospho='u') + BRAF(AA600='E', RBD=ANY, raf=2, rafi=None) % BRAF(RBD=None, raf=2) >>
         MEK(phospho='p') + BRAF(AA600='E', RBD=ANY, raf=2, rafi=None) % BRAF(RBD=None, raf=2), 
         catalyze_RAFrafiNone_MEK_p_kcat)
    Rule('BRAFV600E_phosphorylates_MEK_bound4', 
         MEK(phospho='u') + BRAF(AA600='E', RBD=ANY, raf=2, rafi=None) % CRAF(RBD=None, raf=2) >> 
         MEK(phospho='p') + BRAF(AA600='E', RBD=ANY, raf=2, rafi=None) % CRAF(RBD=None, raf=2), 
         catalyze_RAFrafiNone_MEK_p_kcat)
    Rule('MEK_is_dephosphorylated', 
         MEK(phospho='p') >> 
         MEK(phospho='u'), catalyze_PP2A_MEK_u_kcat)
    Rule('pMEK_phosphorylates_ERK', 
         ERK(phospho='u') + MEK(phospho='p') >> 
         ERK(phospho='p') + MEK(phospho='p'), catalyze_MEKmekiNone_phosphop_ERK_p_kcat)
    Rule('DUSP_dephosphorylates_ERK', 
         ERK(phospho='p') + DUSP() >> 
         ERK(phospho='u') + DUSP(), catalyze_DUSP_ERK_u_kcat)
    Rule('synthesis_DUSP', 
         None >> DUSP(), 
         synthesize_ERKphosphop_DUSP_ksyn)
    Rule('basal_degradation_DUSP', 
         DUSP() >> None, 
         synthesize_ERKphosphop_DUSP_kdeg, delete_molecules=True)
    Rule('activation_RAS', 
         RAS(state='gdp') | RAS(state='gtp'), 
         inhibition_ERKphosphop_RAS_kmod, activation_RAS_kcat)

    EnergyPattern('ep_BRAF_BRAF_mod_RAS_double', RAS(raf=2, state='gtp') % BRAF(RBD=2, raf=1) % BRAF(RBD=3, raf=1) % RAS(raf=3, state='gtp'), ep_RAF_RAF_mod_RASstategtp_double_ddG)
    EnergyPattern('ep_BRAF_CRAF_mod_RAS_double', RAS(raf=2, state='gtp') % BRAF(RBD=2, raf=1) % CRAF(RBD=3, raf=1) % RAS(raf=3, state='gtp'), ep_RAF_RAF_mod_RASstategtp_double_ddG)
    EnergyPattern('ep_CRAF_CRAF_mod_RAS_double', RAS(raf=2, state='gtp') % CRAF(RBD=2, raf=1) % CRAF(RBD=3, raf=1) % RAS(raf=3, state='gtp'), ep_RAF_RAF_mod_RASstategtp_double_ddG)
    EnergyPattern('ep_bind_RASstategtp_BRAF', RAS(raf=1, state='gtp') % BRAF(RBD=1), bind_RASstategtp_RAF_dG)
    EnergyPattern('ep_bind_RASstategtp_CRAF', RAS(raf=1, state='gtp') % CRAF(RBD=1), bind_RASstategtp_RAF_dG)

    Initial(RAS(raf=None, state='gdp'), initRAS)
    Initial(MEK(phospho='u'), initMEK)
    Initial(ERK(phospho='u'), initERK)

extend_model();

## Importing data in PEtab Format

With the model at hand, the next step for model calbitration is defining the training data. We will implement this in PEtab, which simplifies the definition of multiple experimental conditions. The petab specification of a calibration problem consists of the tables describing model observables, experimental measurements, experimental conditions and model parameters. Additionally, tables describing the visualization of simulations and data may also be included. In the following we will guide the reader through the definition

## Observables

We will start the petab definition by specifying the model observables. Observables define the the model quantities that were measured experimentally, here pMEK and pERK. We add respective observables to the model using pysb Observables:

In [11]:
Observable('pMEK_obs', MEK(phospho='p'))
Observable('pERK_obs', ERK(phospho='p'));

These observables quantify all MEK and ERK molecules that are phosphorylated on the site `phospho`, which accounts for phosphorylation on TS on MEK and on ERK.

The model quantifies pMEK and pERK in concentrations, but measurements are noise corrupted, measured in fluorescence intensity and also include background fluorescence. As the scaling between concentrations and intensity and the amount of background signal is known, we include scaling and offset parameters in the petab observable definition. To account for noise corruption, we specify a single noise parameter as noise Formula, which corresponds to Gaussian noise distribution with the respective parameter as standard deviation. The prefix `noiseParameter1` indicates that the value of the parameter will be provided in the respective column of the measurements table.

In [12]:
import petab
import pandas as pd
observable_table = pd.DataFrame([
    {
        petab.OBSERVABLE_ID: obs_id,
        petab.OBSERVABLE_FORMULA: f'{obs_id}_scale*{obs_id}_obs + {obs_id}_offset',
        petab.NOISE_FORMULA: f'noiseParameter1_{obs_id}'
    } for obs_id in ['pMEK', 'pERK']
]).set_index(petab.OBSERVABLE_ID)
observable_table

,observableFormula,noiseFormula
observableId,,
pMEK,pMEK_scale*pMEK_obs + pMEK_offset,noiseParameter1_pMEK
pERK,pERK_scale*pERK_obs + pERK_offset,noiseParameter1_pERK


## Measurements and Conditions
As experimental measurement, we will load one of the datasets from REF, which contains the response of A375 cells cultured for 24h at different concentrations of multiple RAF and MEK inhibitors. The response is measured as normalized phospho-MEK (pMEK) and phospho-ERK (pERK) abundances in both EGF stimulated and unstimulated cells after 5 minutes.

In [13]:
import synapseclient as sc
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 6)

syn = sc.Synapse()
syn.login(silent=True);
data_df = pd.read_csv(syn.get('syn22804081').path)
data_df


UPGRADE AVAILABLE

A more recent version of the Synapse Client (2.4.0) is available. Your version (2.3.1) can be upgraded by typing:
    pip install --upgrade synapseclient

Python Synapse Client version 2.4.0 release notes

https://python-docs.synapse.org/build/html/news.html



,Cell_Line,Vemurafenib,Dabrafenib,...,Std.7,Std.8,Std.9
0,A375,-1,0.0,...,0.009442,0.019582,0.007556
1,A375,-1,0.0,...,0.111256,0.079383,0.141893
...,...,...,...,...,...,...,...
54,A375,0,0.1,...,0.007028,0.018578,0.024706
55,A375,0,0.1,...,0.014206,0.046046,0.117991


The data contains the dose response to multiple RAF an MEK inhibitors as well as respective combinations. However, to minimize computation time requirements, we will only consider data involving the two RAFis vemurafenib and dabrafenib. The model itself only contains generic `RAFi` parameters and species, which we can now map to specific inhibitors using experimental conditions. This permits the simultaneous estimation of inhibitor specific kinetic rates in conjunction with all other model parameters. Such an multi-experiment setup can improve parameter identifiability. 

In the following, we will simulatenously generate the condition and measurement table. The condition table describes the necessary information about experiment, i.e., what drug was used, encoded in parameter mappings, and what concentrations were applied. The measurement table describes the measurement itself as well as additional information such as under which experimental conditions the data was measured.

The data is encoded in a matrix format, while PEtab requires a long format. Thus, additional processing is necessary. Briefly, the code below extracts all data points and sets what model observable they belong to, the value of the measurement, the time of measurement, the noise parameter, information about the experimental condition as well as a dataset ID, which is only used for plotting.

For the EGF stimulation condition, it is important to consider the experimental setup, as cells are subjected to 24h of pretreatment with drugs. We here assume that this is sufficient time for the system to reach a steady-state. To describe such an experimental setup in PEtab, it is necessary to reference the respective experimental condition in the condition table by specifying the respective `petab.CONDITION_ID` in the `petab.PREEQUILIBRATION_CONDITION_ID` column. The condition for the actual experiment can then be specified in the `petab.SIMULATION_CONDITION_ID`.

To enable the visualization of data and fitting results, we here also specify the `petab.DATASET_ID`. Contrary to what the name may suggests, this does not influence the actual fitting, but serves as grouping identifier during plotting.

In [14]:
RAFis = ['Vemurafenib', 'Dabrafenib', 'PLX8394', 'LY3009120', 'AZ_628']
MEKis = ['Cobimetinib', 'Trametinib', 'Selumetinib', 'Binimetinib', 'PD0325901']


conditions = dict()


def format_petab(row):
    suffixes = [''] + [f'.{idx}' for idx in range(1,10)]
    datapoints = []
    if row[MEKis].any():
        return datapoints
    # loop over columns of the data matrix
    for suffix in suffixes:
        # extract data
        datapoint = {
            petab.OBSERVABLE_ID: 'pMEK' if row.pMEK else 'pERK',
            petab.MEASUREMENT: row[f'Mean{suffix}'],
            petab.TIME: row['Time_EGF'],
            petab.NOISE_PARAMETERS: row[f'Std{suffix}'],
        }
        # extract condition information
        if (row[RAFis] != 0).sum() != 1 or ((row['Vemurafenib'] == 0) and (row['Dabrafenib'] == 0)):
            continue

        # find first nonzero
        rafi = RAFis[(row[RAFis] != 0).argmax()]

        # extract drug concentration
        drug_conc = row[f'Concentration (uM){suffix}'] if row[rafi] == -1 else row[rafi]
        # condition id must be sanitzed, must match '^[a-zA-Z]+[\w_]*$'
        drug_str = f'{rafi}_{drug_conc}'.replace('.', '_').replace('-', '_')

        condition_str = drug_str + (
            f'__EGF_{row["EGF"]}' if row['EGF'] > 0 else ''
        )
        condition = {
            petab.CONDITION_ID: condition_str,'RAFi_0': drug_conc,
            'EGF_0': row["EGF"], 'bind_RAFi_RAF_Ea': f'bind_{rafi}_RAF_Ea',
            'bind_RAFi_RAF_dG': f'bind_{rafi}_RAF_dG',
            'ep_RAF_RAF_mod_RAFi_single_ddG': f'ep_RAF_RAF_mod_{rafi}_single_ddG',
            'ep_RAF_RAF_mod_RAFi_double_ddG': f'ep_RAF_RAF_mod_{rafi}_double_ddG'
        }
        # set baseline for datapoint
        datapoint[petab.PREEQUILIBRATION_CONDITION_ID] = drug_str
        # set id for condition and datapoint
        datapoint[petab.SIMULATION_CONDITION_ID] = condition_str
        datapoint[petab.DATASET_ID] =  ('EGF__' if row['EGF'] > 0 else 'ctrl__') + rafi
        condition[petab.CONDITION_ID] = condition_str
        
        datapoints.append(datapoint)
        if condition_str not in conditions:
            conditions[condition_str] = condition
            
    return datapoints

measurement_table = pd.DataFrame([
    d
    for ir, row in data_df.iterrows()
    for d in format_petab(row)
])
condition_table = pd.DataFrame(conditions.values()).set_index(petab.CONDITION_ID);

In the above code we directly used experimentally measured standard deviations, which can be noisy as they are often computed from a small number of biological or technical replicates. As the magnitude of standard deviations determines the importance of datapoints during calibration, outliers can lead to overemphasis of individual datapoints resulting in poor fits to the overall data. To avoid such issues, we use the same averaged standard deviations for all datapoints of each observable.

In [15]:
import numpy as np
for group, frame in measurement_table.groupby(petab.OBSERVABLE_ID):
    measurement_table.loc[measurement_table[petab.OBSERVABLE_ID]==group, petab.NOISE_PARAMETERS] = \
        np.sqrt(frame[petab.NOISE_PARAMETERS].apply(np.square).mean())

In [16]:
measurement_table

,observableId,measurement,time,...,preequilibrationConditionId,simulationConditionId,datasetId
0,pERK,0.969789,0.0833,...,Vemurafenib_0_000210819,Vemurafenib_0_000210819,ctrl__Vemurafenib
1,pERK,0.944607,0.0833,...,Vemurafenib_0_000666667,Vemurafenib_0_000666667,ctrl__Vemurafenib
...,...,...,...,...,...,...,...
78,pMEK,1.172071,0.0833,...,Dabrafenib_0_210818511,Dabrafenib_0_210818511__EGF_100,EGF__Dabrafenib
79,pMEK,1.091548,0.0833,...,Dabrafenib_0_666666667,Dabrafenib_0_666666667__EGF_100,EGF__Dabrafenib


In [17]:
condition_table

,RAFi_0,EGF_0,bind_RAFi_RAF_Ea,bind_RAFi_RAF_dG,ep_RAF_RAF_mod_RAFi_single_ddG,ep_RAF_RAF_mod_RAFi_double_ddG
conditionId,,,,,,
Vemurafenib_0_000210819,0.000211,0,bind_Vemurafenib_RAF_Ea,bind_Vemurafenib_RAF_dG,ep_RAF_RAF_mod_Vemurafenib_single_ddG,ep_RAF_RAF_mod_Vemurafenib_double_ddG
Vemurafenib_0_000666667,0.000667,0,bind_Vemurafenib_RAF_Ea,bind_Vemurafenib_RAF_dG,ep_RAF_RAF_mod_Vemurafenib_single_ddG,ep_RAF_RAF_mod_Vemurafenib_double_ddG
...,...,...,...,...,...,...
Dabrafenib_0_210818511__EGF_100,0.210819,100,bind_Dabrafenib_RAF_Ea,bind_Dabrafenib_RAF_dG,ep_RAF_RAF_mod_Dabrafenib_single_ddG,ep_RAF_RAF_mod_Dabrafenib_double_ddG
Dabrafenib_0_666666667__EGF_100,0.666667,100,bind_Dabrafenib_RAF_Ea,bind_Dabrafenib_RAF_dG,ep_RAF_RAF_mod_Dabrafenib_single_ddG,ep_RAF_RAF_mod_Dabrafenib_double_ddG


## Parameters
The last table necessary for model training is the parameter table. This table describes all free parameters of the model, whether they have to be estimated as well as, parameter scale, nominal values, boundary values and information about priors.

Both parameter boundaries and priors can be used to restrict the search space to biologically plausible parameter values, or prevent numerical issues in regions of parameter space where numerical integration of model equations is challenging. Parameter boundaries set hard threshold that cannot be exceeded, while parameter boundaries are soft constraints that also regularize the calibration. While priors are necessary for Bayesian analysis such as parameter sampling, they are not generally required for optimization based approaches.

For most models, free parameters will primarily occur in the model specification. But, all of previously described tables may introduce new parameters, e.g., in noise or observable formulas or condition. Moreover, the tables may fix the values of both model parameters and newly introduced parameters to numerical values, rendering them non-estimatable. As the parameter table must only describe free parameters, some processing is usually necessary to identify the set of parameters that require specification in the parameter table.

In the following, we will construct a list of `condition_pars`, which are either mapped to  condition specific parameters, such as the generic RAFi energy parameters, or set to fixed values, such as the `RAFi_0` initial concentration. To construct the list of free parameters, we identify all parameters in the model that are not contained in these `condition_pars`, and add all newly introduced condition specific parameters in the condition table as well as the four offset and scaling parameters that were introduced in the observables table.

In [18]:
import numpy as np
condition_pars = [
    par.name
    for par in model.parameters
    if par.name in condition_table.columns
]

free_parameters = [
    par.name for par in model.parameters
    if par.name not in condition_pars
] + [
    name
    for par in condition_pars
    for name in np.unique(condition_table[par])
    if isinstance(name, str)
] + [
    'pMEK_offset','pMEK_scale', 'pERK_offset', 'pERK_scale'
]

All of the parameters in the model follow a consistent naming scheme, which makes it easier to programmatically specify parameter boundaries, scales and whether parameters are to be estimated. Multiple studies have demonstrated that parameter estimation is more efficient when estimating parmaters on a logarithmic scale. The thermodynamic formulation we employed for model construction relies on an exponential dependency between energies and kinetetic rates, which complements these practical insights with a theoretical foundation why model parameters live on a logarithmic scale. However, this also suggests that energetic parameters, in contrast to kinetic rates, should be estimated on a linear scale.

The upper and lower boundaries should ideally be set to biologically plausible values. However, most models employ simplified descriptions of the underlying biochemical processes, which means that model parameters may no longer have a one-to-one correspondance to true biochemical constants and plausible parameter ranges may be difficult to derive. Moreover, plausible parameter ranges may not be known for all parameters. Therefore, in many applications parameter boundaries are initially based on educated guesses and then refined based on estimation results such that optima are contained in the specified ranges and integration failures are not too frequent. Therefore, the boundaries presented here are based on the values used in the original publication and where then refined to work with the simplified model.

Similar to parameter boundaries, nominal values can be difficult to derive. However, the numerical values are only important for model calibration when respective parameters are not estimated. In this example, we do not estimate the initial concentrations and $\phi$ parameters and only set the nominal parameter values of those parameters to non-trivial values. The initial concentrations are set to approximate molecular counts as measured in the original publication and $\phi$ parameters are all set to 1, meaning that free energy differences only affect the reverse rate.


In [19]:
lbs = {
    'Ea': -10, 'dG': -10, 'ddG': -10, 'phi': 0, 'offset': 5e-2,
    'scale': 1e0, 'eq': 1e1, '0': 1e3, 'kcat': 1e-2, 'gexpslope': 1e0,
    'kdeg': 1e-3, 'kM': 1e-3, 'kcatr': 1e-8, 'koff': 1e-5,
}
ubs = {
    'Ea': 10, 'dG': 10, 'ddG': 10, 'phi': 1, 'offset': 2e-1,
    'scale': 1e6, 'eq': 1e5, '0': 1e5, 'kcat': 1e5, 'gexpslope': 1e6,
    'kdeg': 1e2, 'kM': 1e1, 'kcatr': 1e0, 'koff': 1e0,
}

initials = {
    'BRAF_0': 1e3, 'CRAF_0': 1e4, 'MEK_0': 1e5, 'ERK_0': 1e5, 'RAS_0': 5e4,
}


parameter_table = pd.DataFrame([
    {
        petab.PARAMETER_ID: par,
        petab.PARAMETER_SCALE: petab.LIN 
        if par.endswith(('_Ea', '_dG', '_ddG', '_phi')) else petab.LOG10,
        petab.LOWER_BOUND: lbs[par.split('_')[-1]],
        petab.UPPER_BOUND: ubs[par.split('_')[-1]],
        petab.NOMINAL_VALUE: initials[par] if par.endswith('_0') else 1.0,
        petab.ESTIMATE: False if par.endswith(('_phi', '_0')) else True,
        
    } for par in free_parameters
]).set_index(petab.PARAMETER_ID)

parameter_table

,parameterScale,lowerBound,upperBound,nominalValue,estimate
parameterId,,,,,
BRAF_0,log10,1000.00,100000.0,1000.0,False
CRAF_0,log10,1000.00,100000.0,10000.0,False
...,...,...,...,...,...
pERK_offset,log10,0.05,0.2,1.0,True
pERK_scale,log10,1.00,1000000.0,1.0,True


## SBML export
PEtab expects a model in SBML format, but we constructed the model in the pysb format. Therefore, we need to export the model in SBML format.

In [20]:
import libsbml
from pysb.export import export
sbml_reader = libsbml.SBMLReader()
sbml_doc = sbml_reader.readSBMLFromString(export(model, 'sbml'))
sbml_model = sbml_doc.getModel()

In the SBML model, pysb observables are exported as SBML parameters with assignment rules, which programmatically generated identifiers that do not agree with the observable names. As we specified observable formulas based on the respective pysb names, we have to change the identifiers of these parameter to match the pysb names such that the respective references are recognized during petab import.

In [21]:
# rename pysb exported observables
sbml_model.getParameter('__obs1').setId('pMEK_obs')
sbml_model.getAssignmentRuleByVariable('__obs1').setVariable('pMEK_obs')
sbml_model.getParameter('__obs2').setId('pERK_obs')
sbml_model.getAssignmentRuleByVariable('__obs2').setVariable('pERK_obs');

As last step, we create a petab problem using all of the previously constructed table as well as the exported SBML model. As problem construction was a lengthy error-prone process, we performt a static analysis of the petab problem using the petab library provided linter.

In [22]:
import petab
petab_problem = petab.Problem(
    sbml_model=sbml_model, sbml_reader=sbml_reader, sbml_document=sbml_doc,
    measurement_df=measurement_table, condition_df=condition_table,
    observable_df=observable_table, parameter_df=parameter_table
)
petab.lint.lint_problem(petab_problem)

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s6' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s7' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initial

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s22' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s23' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initi

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s38' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s39' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initi

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s54' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s55' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initi

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s70' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s71' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initi

...

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s42' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s43' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initi

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s58' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s59' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initi

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s74' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s75' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initi

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s90' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initialAssignment> or <assignmentRule>.

libSBML Warning (Modeling practice): As a principle of best modeling practice, the <species> should set an initial value (amount or concentration) rather than be left undefined. Doing so improves the portability of models between different simulation and analysis systems, and helps make it easier to detect potential errors in models.
 The <species> with the id '__s91' does not have an 'initialConcentration' or 'initialAmount' attribute, nor is its initial value set by an <initi

True

## Calibrating the model in pyPESTO
Model calibration based on PEtab format is supported by a growing number of tools including COPASI and data2dynamics. Here we used the python based calibration tool pyPESTO for calibration. pyPESTO uses AMICI for model import and simulation, constructs an objective function and provides an interface to various different optimizers. 

In [23]:
import pypesto
import pypesto.petab
importer = pypesto.petab.PetabImporter(petab_problem,
                                       model_name=model.name)

pypesto_problem = importer.create_problem(force_compile=True)

Compiling amici model to folder /Users/ffroehlich/Documents/HMS/energy_modeling_chapter/amici_models/thermo_raf.
2021-08-05 11:54:14.744 - amici.petab_import - INFO - Importing model ...
2021-08-05 11:54:14.746 - amici.petab_import - INFO - Model name is 'thermo_raf'.
Writing model code to '/Users/ffroehlich/Documents/HMS/energy_modeling_chapter/amici_models/thermo_raf'.
2021-08-05 11:54:14.762 - amici.petab_import - INFO - Species: 98
2021-08-05 11:54:14.763 - amici.petab_import - INFO - Global parameters: 117
2021-08-05 11:54:14.763 - amici.petab_import - INFO - Reactions: 360
2021-08-05 11:54:14.835 - amici.petab_import - INFO - Observables: 2
2021-08-05 11:54:14.836 - amici.petab_import - INFO - Sigmas: 2
2021-08-05 11:54:14.840 - amici.petab_import - DEBUG - Adding output parameters to model: OrderedDict([('pMEK_offset', None), ('pMEK_scale', None), ('pERK_offset', None), ('pERK_scale', None), ('noiseParameter1_pMEK', None), ('noiseParameter1_pERK', None)])
2021-08-05 11:54:14.841

2021-08-05 11:54:45.153 - amici.ode_export - INFO - Finished computing dxdotdp_explicit         +++ (2.31E-02s)
2021-08-05 11:54:45.167 - amici.ode_export - INFO - Finished writing dxdotdp_explicit.cpp        ++ (4.06E-02s)
2021-08-05 11:54:45.187 - amici.ode_export - INFO - Finished running smart_jacobian           +++++ (4.59E-04s)
2021-08-05 11:54:45.194 - amici.ode_export - INFO - Finished simplifying dydx                 +++++ (7.41E-04s)
2021-08-05 11:54:45.195 - amici.ode_export - INFO - Finished computing dydx                    ++++ (1.16E-02s)
2021-08-05 11:54:45.246 - amici.ode_export - INFO - Finished running smart_jacobian           +++++ (3.94E-02s)
2021-08-05 11:54:45.651 - amici.ode_export - INFO - Finished running smart_multiply           +++++ (4.00E-01s)
2021-08-05 11:54:45.662 - amici.ode_export - INFO - Finished simplifying dydw                 +++++ (3.21E-03s)
2021-08-05 11:54:45.663 - amici.ode_export - INFO - Finished computing dydw                    ++++ (4.6

running build_ext
Changed extra_compile_args for unix to ['-Xpreprocessor', '-fopenmp', '-std=c++14']
building 'thermo_raf._thermo_raf' extension
Testing SWIG executable swig4.0... FAILED.
Testing SWIG executable swig3.0... FAILED.
Testing SWIG executable swig... SUCCEEDED.
swigging swig/thermo_raf.i to swig/thermo_raf_wrap.cpp
swig -python -c++ -modern -outdir thermo_raf -I/Users/ffroehlich/Documents/HMS/energy_modeling_chapter/venv/lib/python3.9/site-packages/amici/swig -I/Users/ffroehlich/Documents/HMS/energy_modeling_chapter/venv/lib/python3.9/site-packages/amici/include -o swig/thermo_raf_wrap.cpp swig/thermo_raf.i
Deprecated command line option: -modern. This option is now always on.
creating build
creating build/temp.macosx-10.14-x86_64-3.9
creating build/temp.macosx-10.14-x86_64-3.9/swig
clang -Wno-unused-result -Wsign-compare -Wunreachable-code -fno-common -dynamic -DNDEBUG -g -fwrapv -O3 -Wall -I/Users/ffroehlich/Documents/HMS/energy_modeling_chapter/amici_models/thermo_raf -

Simulations in AMICI are highly customizable and while many steps, including everything that was specified in the the petab problem, are automated by pypesto, not all options are set automatically and some user input may be required. For example, the considered example includes preequilibrations which is automatically handled by pyPESTO and AMICI. However, AMICI supports multiple different methods to compute steadystates as well as respective sensitivities. By default, a combination of Newton's method and simulation is employed to find the steadystate and sensitivites are computed using the Implicit Function Theorem (IFT), which is the most efficient option when applicable. Sensitivity computation via the IFT requires that the Jacobian of the model equations is not singular, which is not the case for models with conservation laws. As we do not account for protein synthesis and degradation, total protein abundances are conserved quantities in the model and the IFT is not applicable. Thus, we here employ a purely simulation based steadystate search by setting the allowed newton steps to 0 and compute the steadystate sensitivities using forward sensitivity analysis by setting the steadystate sensitivity mode accordingly.

In [24]:
import amici
pypesto_problem.objective.amici_solver.setNewtonMaxSteps(0)
pypesto_problem.objective.amici_model.setSteadyStateSensitivityMode(amici.SteadyStateSensitivityMode.simulationFSA)
pypesto_problem.objective.guess_steadystate = False

AMICI also supports different sensitivity analysis methods for the simulation itself. For models with over 100 parameters, it usually makes sense to use adjoint sensitivity analysis for simulation conditions. However, for models that also require preequilibration, the computation time of the simulation and gradient evaluation is dominated by the preequilibration part and the choice of sensitivity method for the simulation part will not have a significant impact on computation time. Accordingly, other considerations, such as the availability of approximate second order information should be taken into account when selecting the sensitivity method. In contrast to forward sensitivity analysis, adjoint sensitivity analysis does not provide an approximation to the Hessian of the objective function, rendering some optimizers inapplicable and potentially reducing the convergence rate of those that remain applicable. Therefore, we here chose forward sensitivty analysis for the simulation part.

In [25]:
pypesto_problem.objective.amici_solver.setSensitivityMethod(amici.SensitivityMethod.forward)

Another set of crucial parameters to tune are integration step limits and tolerances. Lower tolerances generally improve accuracy of the objective function and its derivatives, which influences local convergence rate and success. However, lower tolerances also increases the computation time, so fewer optimization iterations could easily be offsetted by longer compute times per iteration. Moreover, both integration tolerances and step limits influence numerical integrability, which can lead to complications during optimization. For some optimizers, low step limits can prevent optimization from getting stuck in difficult to integrate parameter regions, but if parameters from these regions yield good fits to the data, this can also prevent optimization from reaching good fits. Accordingly, setting adequate values for step limits and tolerances is integral for model calibration. Nevertheless, there are little to no theoretical or practical guidelines on how to choose adequate values. Ranges of $10^{-8}$ to $10^{-12}$ are usually adequate, but most of the time values are chosen and updated empirically. Here we chose tolerances of $10^{-12}$ and a step limit of $10^{5}$ integration steps, which is rather conservative, but ensures adequate local convergence.

In [26]:
pypesto_problem.objective.amici_solver.setAbsoluteTolerance(1e-12)
pypesto_problem.objective.amici_solver.setRelativeTolerance(1e-12)
pypesto_problem.objective.amici_solver.setMaxSteps(int(1e5))

Even with tuned integration tolerances and step limits, integration failures which prevent evaluation of the objective function and it's derivatives is usually inevitable. While some optimizers are able to continue after integration failures by rejecting the respective parameter update, it is impossible to recover from an integration failure during initialization. To still ensure that a sufficient number number of optimization runs is performed, pyPESTO enables resampling of startpoints, which generates new startpoints until the objective function could be successfully evaluated at all starting points. In the following, we activate this behaviour by passing an options object with the respective setting activated.

In [27]:
from pypesto.optimize import OptimizeOptions
optimize_options = OptimizeOptions(startpoint_resample=True)

Selecting an appropriate number of optimization runs can be challenging but should generally be adapted to the downstream analysis. For example, when model selection or uncertainty analysis using profile likelihoods will be performed, it is crucial to ensure that optimization reaches a local optimum and that the local optimum is, or at least yields a fit equivalent to, the global optimum. This can be achieved by running many optimization runs and allocating large computational budgets for each run. For large models, the total computational budget usually is insufficient to achieve these goals and the number of runs and the computational budget has to be limited, with an unknown tradeoff between the two. To deal with this tradeoff, we will here use a two-staged approach where we first run many starts with a low computational budget of 10 minutes and will then select the most promising runs and continue optimization with a longer computational budget. For this purpose, we use the fides optimizer, a high performance trust region optimizer which permits the direct specification of the computational budgets.

In [28]:
from pypesto.optimize import FidesOptimizer
import fides

optimizer = FidesOptimizer(options={fides.Options.MAXTIME: 600})

Now we have everything at hand to run the first calibration stage, where run 100 optimization runs with uniformly sampled starting points. The calibration is parallelized using pyPESTOs multithreading engine with 4 threads.

In [ ]:
from pypesto.optimize import minimize
result = minimize(
    pypesto_problem, optimizer, n_starts=100, options=optimize_options,
    startpoint_method=pypesto.startpoint.uniform,
    engine=pypesto.engine.MultiThreadEngine(4)
)

[Warning] AMICI:CVODES:CVode:WARNING: AMICI ERROR: in module CVODES in function CVode : Internal t = 351.988 and h = 2.22949e-14 are such that t + h = t on the next step. The solver will continue anyway. 
[Warning] AMICI:CVODES:CVode:WARNING: AMICI ERROR: in module CVODES in function CVode : Internal t = 351.988 and h = 2.22949e-14 are such that t + h = t on the next step. The solver will continue anyway. 
[Warning] AMICI:CVODES:CVode:WARNING: AMICI ERROR: in module CVODES in function CVode : Internal t = 351.988 and h = 2.22949e-14 are such that t + h = t on the next step. The solver will continue anyway. 
[Warning] AMICI:CVODES:CVode:WARNING: AMICI ERROR: in module CVODES in function CVode : Internal t = 351.988 and h = 2.22949e-14 are such that t + h = t on the next step. The solver will continue anyway. 
[Warning] AMICI:CVODES:CVode:WARNING: AMICI ERROR: in module CVODES in function CVode : Internal t = 351.988 and h = 2.22949e-14 are such that t + h = t on the next step. The solve

[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AM

[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AM

[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 8/96 in fxdot!
[Warning] AM

[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!

[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!

[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!
[Warning] AMICI:Inf: AMICI encountered an Inf value at index 38/96 in fxdot!

Performing parallel task execution on 4 threads.
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 11064.43it/s]Executing task 1.

Executing task 2.
Executing task 3.
[Warning] AMICI:simulation: AMICI simulation failed:
Steady state computation failed. First run of Newton solver failed: No convergence was achieved. Simulation to steady state failed: No convergence was achieved. Second run of Newton solver failed: No convergence was achieved.
Error occurred in:
0          0x1350ae383 amici::SteadystateProblem::handleSteadyStateFailure(amici::Solver const*, amici::Model*) + 531
1          0x1350ad65c amici::SteadystateProblem::findSteadyState(amici::Solver*, amici::NewtonSolver*, amici::Model*, int) + 332
2          0x1350ad3be amici::SteadystateProblem::workSteadyStateProblem(amici::Solver*, amici::Model*, int) + 254
3          0x13508eeb1 amici::AmiciApplication::runAmiciSimulation(amici::Solver

To assess whether the quality of the calibration, we will now use petabs visualization feature. The visualization instructions are again specified in table, where, in the case of line plots, every row corresponds to a single pair of connected lines for data an simulation. Here we make use of the of the `petab.DATASET_ID` values that we specified in the measurements table. In the following we will generate seperate plots for each observable and each inhibitor, where every plot shows the dose response in the EGF stimulated and unstimulated condition.

In [ ]:
visualization_table = pd.DataFrame([
    {
        petab.PLOT_ID: f'{rafi}_{obs}',
        petab.PLOT_TYPE_SIMULATION: petab.LINE_PLOT,
        petab.PLOT_TYPE_DATA: petab.MEAN_AND_SD,
        petab.DATASET_ID: condition,
        petab.X_VALUES: 'RAFi_0',
        petab.Y_VALUES: obs,
        petab.X_SCALE: petab.LOG10,
        petab.X_LABEL: rafi,
        petab.Y_LABEL: obs,
        petab.LEGEND_ENTRY: condition.split('__')[0]
    }
    for rafi in RAFis
    for obs in observable_table.index
    for condition in measurement_table[petab.DATASET_ID].unique()
    if condition.split('__')[1] == rafi
])

In [ ]:
visualization_table

While experimental data for visualization is already available in the measurements table, simulations of the calibrated model still have to be computed. To generate model simulations, we extract the pre-calibrated parameter values for the best fit of the first calibration round. As optimization results are sorted in increasing order of objective function values, this can be achieved by taking the result at the first index. Simulation results can be extracted using the pypesto objective and the amici function `amici.petab_objective.rdatas_to_simulation_df`. As the results contains the full parameter vector including parameter values that were not estimated, but the objective only expects estimated parameters, the parameters need to be subsetted according to the free indices. 

In [ ]:
x = pypesto_problem.get_reduced_vector(result.optimize_result.list[0]['x'],
                                       pypesto_problem.x_free_indices)
simulation = pypesto_problem.objective(x, return_dict=True)
simulation_df = amici.petab_objective.rdatas_to_simulation_df(
    simulation['rdatas'],
    model=pypesto_problem.objective.amici_model,
    measurement_df=measurement_table,
)

In [ ]:
simulation_df = simulation_df.sort_values([petab.OBSERVABLE_ID, petab.PREEQUILIBRATION_CONDITION_ID, 
                                           petab.SIMULATION_CONDITION_ID]).reset_index()
measurement_table = measurement_table.sort_values([petab.OBSERVABLE_ID, petab.PREEQUILIBRATION_CONDITION_ID, 
                                                   petab.SIMULATION_CONDITION_ID]).reset_index()

We can now visualize both data and simulation using `petab.visualize.plot_data_and_simulation`.

In [ ]:
import petab.visualize
petab.visualize.plot_data_and_simulation(
    exp_data=measurement_table,
    exp_conditions=condition_table,
    sim_data=simulation_df,
    vis_spec=visualization_table
);

The visualization suggests, that model simulations take values that are similar to the data they still show qualitatively different behaviour. This suggests that the calibrated parameter values are an adequate first guess,  that this is an adequate starting point for further refinement.

Before we proceed with the second calibration round, we can check the accuracy of the gradient at the pre-calibrated parameter values. 

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pypesto_problem.objective.x_names = [pypesto_problem.objective.x_names[ix] for ix in pypesto_problem.x_free_indices]
pypesto_problem.objective.check_grad_multi_eps(x, multi_eps=[1e-1, 1e-3, 1e-5],
                                               verbosity=0)

The function computes forward (`fd_c`), backward (`fd_b`) and central (`fd_c`) finite differences and compares them to the sensitivity based gradient (`grad`), using the optimal step size `eps` among the provided values. The results show a low error in the finite difference approximation (`fd_err`) indicating accurate evaluation of the objective function. Moreover, the small absolute (`abs_err`) and relative difference (`rel_err`) between sensitivity based gradient and finite difference approximation indicates accurate evaluation of the gradient. The large magnitude of the gradient suggest that we did not yet reach a local optimum and that further optimization will likely improve agreement between data and simulation.

We will now perfom this refinement by instantiating a new optimizer with a larger computational budget of one hour

In [ ]:
optimizer_fine = FidesOptimizer(options={fides.Options.MAXTIME: 3600})

To initialize the second calibration round at the pre-calibrated parameter values, we extract the respective values from the results object and set them as guesses in the pypesto problem.

In [ ]:
n_refine = 10
pypesto_problem.x_guesses_full = np.vstack(result.optimize_result.get_for_key('x')[:n_refine])

For startpoint generation, pypesto will first start optimization runs at the provided parameter guesses and only sample new startpoint if more optimization runs than guesses are requested. As for the first calibration run, we run the calibration using the `minimize` function.

In [ ]:
result_refined = minimize(
    pypesto_problem, optimizer_fine, n_starts=n_refine,
    engine=pypesto.engine.MultiThreadEngine(4)
)

As for the pre-calibration, we now visualize the results using `petab` and the previously constructed visualization table.

In [ ]:
x = pypesto_problem.get_reduced_vector(result_refined.optimize_result.list[0]['x'],
                                       pypesto_problem.x_free_indices)
simulation = pypesto_problem.objective(x, return_dict=True)
simulation_df = amici.petab_objective.rdatas_to_simulation_df(
    simulation['rdatas'],
    model=importer.create_model(),
    measurement_df=measurement_table,
)

simulation_df = simulation_df.sort_values([petab.OBSERVABLE_ID, petab.PREEQUILIBRATION_CONDITION_ID, 
                                           petab.SIMULATION_CONDITION_ID]).reset_index()
measurement_table = measurement_table.sort_values([petab.OBSERVABLE_ID, petab.PREEQUILIBRATION_CONDITION_ID, 
                                                   petab.SIMULATION_CONDITION_ID]).reset_index()

petab.visualize.plot_data_and_simulation(
    exp_data=measurement_table,
    exp_conditions=condition_table,
    sim_data=simulation_df,
    vis_spec=visualization_table
);

These plots now indicate good qualitative and quantitative agreement between experimental data and model simulations. As a last step of the analysis, we now compare the objective function values across both calibration rounds using a waterfall plot.

In [ ]:
from pypesto.visualize import waterfall
waterfall(results=[result,result_refined],
          legends=['pre-calibration', 'calibration']);

The waterfall plot shows that the 